<a href="https://colab.research.google.com/github/sribarrow/dissertation/blob/master/Copy_of_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import keras
from keras import backend as K
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import LSTM, Conv2D, MaxPooling2D, Dropout
from keras.layers import Dense, Activation, Flatten
from keras import optimizers, regularizers

Using TensorFlow backend.


In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from tensorflow.python.client import device_lib

In [4]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13138579975080691054, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7331140261920052253
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10872227931879300993
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14892338381
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13499304787938517774
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [11]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
# with open('/gdrive/My Drive/foo.txt', 'w') as f:
#   f.write('Hello Google Drive!')
# !cat '/gdrive/My Drive/foo.txt'

In [0]:
# config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 1} ) 
#  sess = tf.Session(config=config) 
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [6]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [7]:
from tensorflow.core.protobuf import config_pb2
sess.run(c, options=config_pb2.RunOptions(
        report_tensor_allocations_upon_oom=True))

array([[22., 28.],
       [49., 64.]], dtype=float32)

In [0]:
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [0]:
train_path = '/gdrive/My Drive/raag_images/train'
valid_path = '/gdrive/My Drive/raag_images/check'
test_path = '/gdrive/My Drive/raag_images/test'

In [13]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(300,300), classes=['RaagM','RaagS'],  batch_size=32)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path,  target_size=(300,300), classes=['RaagM','RaagS'],  batch_size=4)
test_batches = ImageDataGenerator().flow_from_directory(test_path,  target_size=(300,300), classes=['RaagM','RaagS'],  batch_size=10)

Found 576 images belonging to 2 classes.
Found 26 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [14]:
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [21]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=(300,300,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 300, 300, 32)      896       
_________________________________________________________________
activation_8 (Activation)    (None, 300, 300, 32)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 298, 298, 64)      18496     
_________________________________________________________________
activation_9 (Activation)    (None, 298, 298, 64)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 149, 149, 64)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 147, 147, 64)      36928     
_________________________________________________________________
activation_10 (Activation)   (None, 147, 147, 64)      0         
__________

In [22]:
STEP_SIZE_TRAIN=train_batches.n//train_batches.batch_size
print(STEP_SIZE_TRAIN)
STEP_SIZE_VALID=valid_batches.n//valid_batches.batch_size
print(STEP_SIZE_VALID)


18
6


In [23]:
model.fit_generator(train_batches, steps_per_epoch=STEP_SIZE_TRAIN, epochs=10, validation_data=valid_batches, validation_steps=STEP_SIZE_VALID)

Epoch 1/10
18/18 [==============================] - 7s 413ms/step - loss: 7.8739 - acc: 0.5069 - val_loss: 8.7917 - val_acc: 0.4545
Epoch 2/10
18/18 [==============================] - 6s 308ms/step - loss: 8.2829 - acc: 0.4861 - val_loss: 8.7917 - val_acc: 0.4545
Epoch 3/10
18/18 [==============================] - 6s 310ms/step - loss: 7.7792 - acc: 0.5174 - val_loss: 8.7306 - val_acc: 0.4583
Epoch 4/10
18/18 [==============================] - 6s 313ms/step - loss: 8.2549 - acc: 0.4878 - val_loss: 8.7917 - val_acc: 0.4545
Epoch 5/10
18/18 [==============================] - 6s 314ms/step - loss: 8.2269 - acc: 0.4896 - val_loss: 5.1285 - val_acc: 0.6818
Epoch 6/10
18/18 [==============================] - 6s 314ms/step - loss: 7.9191 - acc: 0.5087 - val_loss: 10.9896 - val_acc: 0.3182
Epoch 7/10
18/18 [==============================] - 6s 312ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 6.5938 - val_acc: 0.5909
Epoch 8/10
18/18 [==============================] - 6s 310ms/step - loss: 8